In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# CropNet: Cassava Disease Detection

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/cropnet_cassava"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/cropnet_cassava.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/cropnet_cassava.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/cropnet_cassava.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub model</a>
  </td>
</table>

이 노트북은 **TensorFlow Hub**의 CropNet [카사바 질병 분류자](https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2) 모델을 사용하는 방법을 보여줍니다. 이 모델은 카사바 잎의 이미지를 *세균성 마름병, 갈색 줄무늬병, 녹색 진드기, 모자이크병, 건강함 또는 알 수 없음*의 6가지 등급 중 하나로 분류합니다.

이 Colab에서는 다음 방법을 보여줍니다.

- **TensorFlow Hub**에서 https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2 모델을 로드합니다.
- **TFDS(TensorFlow 데이터세트)**에서 [cassava](https://www.tensorflow.org/datasets/catalog/cassava) 데이터세트를 로드합니다.
- 카사바 잎의 이미지를 4개의 특징적인 카사바 질병 범주 또는 건강하거나 알려지지 않은 상태로 분류합니다.
- 분류자의 *정확성*을 평가하고 도메인 외부 이미지에 적용했을 때 모델이 얼마나 *강력*한지 확인합니다.

## 가져오기 및 설정

In [ ]:
!pip install matplotlib==3.2.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [ ]:
#@title Helper function for displaying examples
def plot(examples, predictions=None):
  # Get the images, labels, and optionally predictions
  images = examples['image']
  labels = examples['label']
  batch_size = len(images)
  if predictions is None:
    predictions = batch_size * [None]

  # Configure the layout of the grid
  x = np.ceil(np.sqrt(batch_size))
  y = np.ceil(batch_size / x)
  fig = plt.figure(figsize=(x * 6, y * 7))

  for i, (image, label, prediction) in enumerate(zip(images, labels, predictions)):
    # Render the image
    ax = fig.add_subplot(x, y, i+1)
    ax.imshow(image, aspect='auto')
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])

    # Display the label and optionally prediction
    x_label = 'Label: ' + name_map[class_names[label]]
    if prediction is not None:
      x_label = 'Prediction: ' + name_map[class_names[prediction]] + '\n' + x_label
      ax.xaxis.label.set_color('green' if label == prediction else 'red')
    ax.set_xlabel(x_label)

  plt.show()

## 데이터세트

TFDS에서 *cassava* 데이터세트를 불러오겠습니다.

In [ ]:
dataset, info = tfds.load('cassava', with_info=True)

설명과 인용, 사용 가능한 예제 수에 대한 정보 등 데이터세트에 대해 자세히 알아보겠습니다.

In [ ]:
info

*cassava* 데이터세트에는 4가지 질병이 있는 카사바 잎 이미지와 건강한 카사바 잎이 있습니다. 모델은 이러한 모든 등급을 예측할 수 있고, 예측에 확신이 없는 경우에는 여섯 번째 등급인 "알 수 없음"으로 분류합니다.

In [ ]:
# Extend the cassava dataset classes with 'unknown'
class_names = info.features['label'].names + ['unknown']

# Map the class names to human readable names
name_map = dict(
    cmd='Mosaic Disease',
    cbb='Bacterial Blight',
    cgm='Green Mite',
    cbsd='Brown Streak Disease',
    healthy='Healthy',
    unknown='Unknown')

print(len(class_names), 'classes:')
print(class_names)
print([name_map[name] for name in class_names])

모델에 데이터를 공급하기 전에 약간의 전처리가 필요합니다. 모델은 RGB 채널 값이 [0, 1] 범위인 224 x 224 이미지를 예상합니다. 이미지를 정규화하고 크기를 조정하겠습니다.

In [ ]:
def preprocess_fn(data):
  image = data['image']

  # Normalize [0, 255] to [0, 1]
  image = tf.cast(image, tf.float32)
  image = image / 255.

  # Resize the images to 224 x 224
  image = tf.image.resize(image, (224, 224))

  data['image'] = image
  return data

데이터세트의 몇 가지 예를 살펴보겠습니다.

In [ ]:
batch = dataset['validation'].map(preprocess_fn).batch(25).as_numpy_iterator()
examples = next(batch)
plot(examples)

## 모델

TF-Hub에서 분류자를 로드하고 일부 예측값을 가져와서 몇 가지 예에서 모델의 예측값이 어떤지 확인하겠습니다.

In [ ]:
classifier = hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2')
probabilities = classifier(examples['image'])
predictions = tf.argmax(probabilities, axis=-1)

In [ ]:
plot(examples, predictions)

## 평가 및 견고성

일부 데이터세트에서 분류자의 *정확성*을 측정해 보겠습니다. 또한 카사바가 아닌 데이터세트에서 성능을 평가하여 모델의 *견고성*을 확인할 수 있습니다. iNaturalist 또는 콩과 같은 다른 식물 데이터세트의 이미지의 경우, 모델은 거의 항상 *unknown*을 반환해야 합니다.

In [ ]:
#@title Parameters {run: "auto"}

DATASET = 'cassava'  #@param {type:"string"} ['cassava', 'beans', 'i_naturalist2017']
DATASET_SPLIT = 'test' #@param {type:"string"} ['train', 'test', 'validation']
BATCH_SIZE =  32 #@param {type:"integer"}
MAX_EXAMPLES = 1000 #@param {type:"integer"}


In [ ]:
def label_to_unknown_fn(data):
  data['label'] = 5  # Override label to unknown.
  return data

In [ ]:
# Preprocess the examples and map the image label to unknown for non-cassava datasets.
ds = tfds.load(DATASET, split=DATASET_SPLIT).map(preprocess_fn).take(MAX_EXAMPLES)
dataset_description = DATASET
if DATASET != 'cassava':
  ds = ds.map(label_to_unknown_fn)
  dataset_description += ' (labels mapped to unknown)'
ds = ds.batch(BATCH_SIZE)

# Calculate the accuracy of the model
metric = tf.keras.metrics.Accuracy()
for examples in ds:
  probabilities = classifier(examples['image'])
  predictions = tf.math.argmax(probabilities, axis=-1)
  labels = examples['label']
  metric.update_state(labels, predictions)

print('Accuracy on %s: %.2f' % (dataset_description, metric.result().numpy()))

## 자세히 알아보기

- TensorFlow Hub의 모델에 대해 자세히 알아보기: https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2
- [이 모델의 TensorFlow Lite 버전](https://tfhub.dev/google/lite-model/cropnet/classifier/cassava_disease_V1/1)을 사용하여 [ML 키트](https://developers.google.com/ml-kit/custom-models#tfhub)를 통해 휴대전화에서 실행되는 사용자 정의 이미지 분류자를 빌드하는 방법을 알아보세요.